In [1]:
from pyspark.sql import SparkSession
import datetime,os
        # .config("spark.driver.port","4040") \
        # .config("spark.blockManager.port","4041") \
spark =  SparkSession.builder \
        .appName("Upserts Operation")\
        .master("local[*]") \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .config("spark.driver.bindAddress","127.0.0.1") \
        .config("spark.executor.memory","4g") \
        .config("spark.driver.memory","8g") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f'spark session created {spark} at {datetime.datetime.now()}')

# print(jars)
# spark.stop()

25/05/26 11:14:06 WARN Utils: Your hostname, matrix resolves to a loopback address: 127.0.1.1; using 192.168.184.220 instead (on interface wlx3068933f472b)
25/05/26 11:14:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/jarvis/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jarvis/.ivy2/cache
The jars for the packages stored in: /home/jarvis/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1d262e1-b6c2-4099-90c7-345298646dba;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 159ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.io.IOException: Failed to connect to /192.168.184.220:46863
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:294)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:226)
	at org.apache.spark.rpc.netty.NettyRpcEnv.downloadClient(NettyRpcEnv.scala:399)
	at org.apache.spark.rpc.netty.NettyRpcEnv.$anonfun$openChannel$4(NettyRpcEnv.scala:367)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1397)
	at org.apache.spark.rpc.netty.NettyRpcEnv.openChannel(NettyRpcEnv.scala:366)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:709)
	at org.apache.spark.util.Utils$.fetchFile(Utils

25/05/26 11:14:24 ERROR Utils: Uncaught exception in thread executor-heartbeater
java.lang.ExceptionInInitializerError
	at org.apache.spark.metrics.ProcessTreeMetrics$.getMetricValues(ExecutorMetricType.scala:93)
	at org.apache.spark.executor.ExecutorMetrics$.$anonfun$getCurrentMetrics$1(ExecutorMetrics.scala:103)
	at org.apache.spark.executor.ExecutorMetrics$.$anonfun$getCurrentMetrics$1$adapted(ExecutorMetrics.scala:102)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.executor.ExecutorMetrics$.getCurrentMetrics(ExecutorMetrics.scala:102)
	at org.apache.spark.executor.ExecutorMetricsPoller.poll(ExecutorMetricsPoller.scala:82)
	at org.apac

In [ ]:
from pyspark.sql.functions import current_timestamp,lit
# from delta.tables import DeltaTable

read_options={
    "header":True,
    "inferSchema":True
}

customer_master_src = spark.read.options(**read_options).csv('../datasets/customer_master.csv')
customer_master_src = customer_master_src.withColumn("created_at",current_timestamp())\
                .withColumn("updated_at",current_timestamp())\
                .withColumn("current_flag",lit("Y"))
                
# customer_master_src.show(truncate=False)
try:
    customer_master_src.write.format("delta").mode("overwrite").save("../DeltaLake/customer_master")
    print("Write success")
except Exception as e:
    print(f"Write failed : {e}")

Written success


In [1]:
tgt=spark.read.format("delta").load("../DeltaLake/customer_master")
tgt.show(truncate=False)



NameError: name 'spark' is not defined

In [ ]:
'''Perform SCD Type 2 on it'''



In [ ]:
# Time Travel (Query Past Versions)

df_old = spark.read.format("delta").option("versionAsOf", 0).load("delta_tables/customer")
df_old.show()

In [ ]:
 Schema Enforcement and Evolution

    Enforcement: Will error out if new data doesn’t match schema.

    Evolution: You can enable schema evolution when appending.

df_new.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append").save("delta_tables/customer")